#  Data Analysis and Results Collection

This file is for automaticly output all the analysis for all dataset at once 

In [1]:
#!pip install xlwt

In [1]:
#Imports
import math
import librosa
import librosa.display
import IPython.display as ipd

import pyAudioAnalysis
import pydub
import ffmpeg

from pydub import AudioSegment
from pydub.silence import split_on_silence

import numpy as np
import matplotlib.pyplot as plt

import xlwt
from xlwt import Workbook

from numpy import sin, cos, pi, linspace, arange, log10, absolute
from numpy.random import randn
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, welch
from scipy.io.wavfile import read , write
from scipy.signal import spectrogram
from matplotlib import pyplot
from matplotlib.pyplot import ylabel, xlabel
from matplotlib.pyplot import plot, legend, show, grid, figure, savefig#, hold

C:\Users\Dasil\anaconda3\envs\PDS\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
import numpy as np
def real_cepstrum(x, n=None):
# Compute the real cepstrum of a real sequence
#    x : ndarray
#        Real sequence to compute real cepstrum of.
#    n : {None, int}, optional
#        Length of the Fourier transform.
#    Returns
#    -------
#    ceps: ndarray
#        The real cepstrum.

    spectrum = np.abs(np.fft.fft(x, n=n))**2
    ceps = np.fft.ifft(np.log(spectrum))

    return ceps, spectrum

In [3]:
#Zero Crossing Rate

def ZCR(samples, frameSize, overlap):
    wlen = len(samples)
    step = frameSize - overlap
    frameNum = math.ceil(wlen/step)
    zcr = np.zeros((frameNum,1))
    for i in range(frameNum):
        curFrame = samples[np.arange(i*step,min(i*step+frameSize,wlen))]
        #To avoid DC bias, usually we need to perform mean substraction on each frame
        curFrame = curFrame - np.mean(curFrame) #Zero-Justified
        zcr[i] = sum(curFrame[0:-1]*curFrame[1::]<=0)
    return zcr

In [4]:
def spectral_centroid(x, samplerate=44100):
    magnitudes = np.abs(np.fft.rfft(x)) # magnitudes of positive frequencies
    length = len(x)
    freqs = np.abs(np.fft.fftfreq(length, 1.0/samplerate)[:length//2+1]) # positive frequencies
    return np.sum(magnitudes*freqs) / np.sum(magnitudes) # return weighted mean

In [7]:
# Workbook is created
wb = Workbook()

#List of Words
words = ["_chata_0","_chapa_0","_chave_0","_lata_0","_lapa_0","_lava_0","_casa_0","_capa_0","_cave_0","_cata_0","_chuta_0","_chupa_0","_chuva_0","_farta_0","_farpa_0","_farda_0","_ripa_0","_rita_0","_rica_0"]

#List of Dataset versions
d = {"one": 1, "two": 2}
iterable = d.keys()

#List of people
p = ['A','J','N']

# add_sheet is used to create sheet.
sheet1 = wb.add_sheet('Pitch Analysis')
  
sheet1.write(1, 0, 'Andre')
sheet1.write(1, 1, 'PW_spec')
sheet1.write(2, 1, 'Ceps')
sheet1.write(3, 1, 'Avg ZCR')
sheet1.write(4, 1, 'Spec_zen')
sheet1.write(5, 0, 'Joaquin')
sheet1.write(5, 1, 'PW_spec')
sheet1.write(6, 1, 'Ceps')
sheet1.write(7, 1, 'Avg ZCR')
sheet1.write(8, 1, 'Spec_zen')
sheet1.write(9, 0, 'Nuno')
sheet1.write(9, 1, 'PW_spec')
sheet1.write(10, 1, 'Ceps')
sheet1.write(11, 1, 'Avg ZCR')
sheet1.write(12, 1, 'Spec_zen')

h = 2

#Write Colum Names
for x in words:
    for item in iterable:
        val = x+str(d[item])
        sheet1.write(0, h, val)
        h += 1
h=1
w=0
flag = False

for y in words:
    for item in iterable:
        h += 1
        if (w == 9):
            w = 0        
        for x in p:
            #Write file path
            l = str(d[item])
            filename = 'audio/training/'+x+y+l+'.wav'
            
            #Read file path
            fs, audio1 = read(filename)
            f,t,S1 = spectrogram(audio1, fs, window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='spectrum', mode='magnitude')

            #Apply butterworth Filter
            wn = 1500/(fs/2)
            b3,a3 = butter(4, wn)
            audio1_filt = lfilter(b3,a3,audio1)

            #Analyze Audio (With audio Segment)
            song = AudioSegment.from_wav(filename)

            #Divide Audio in two (The old Way)
            #db = -30
            #segments = split_on_silence(song, min_silence_len = 100, silence_thresh = db)
            #while len(segments) != 2:
            #    db = db-1
            #    segments = split_on_silence(song, min_silence_len = 100, silence_thresh = db)
            #    if db < -90:
            #        break
            
            #Divide Audio in two Nuno's way
            results = [1 if item > 350 else 0 for item in audio1_filt]
            
            x1 = 0
            x2 = 0
            x3 = 0
            x4 = 0
            flag = False
            zeros_list = []
            silence_len = 7500
            change_len = 2000
            for i, result in enumerate(results):
                if(flag == False and result == 1):
                    if (x1 == 0):
                        flag = True
                        x1 = i
                    elif (i != x1):
                        flag = True
                        x3 = i
                if(flag == True):
                    if(result == 0):
                        zeros_list.append(result)
                    else:
                        zeros_list = []
                if(flag == True and len(zeros_list) >= silence_len):
                    if (x2 == 0):
                        flag = False
                        x2 = i - len(zeros_list)
                        if(x2-x1 <= change_len):
                            x1 = x2 = 0
                    elif (i - len(zeros_list) != x2):
                        flag = False
                        x4 = i - len(zeros_list)
                        break

            Div_p = ((x3-x2)/2)+x2
            
            
            # Fix-sized segmentation (breaks a signal into non-overlapping segments)
            signal = audio1 / (2**15)
            signal_len = len(signal)
            segment_size_t = 1 # segment size in seconds
            segment_size = segment_size_t * fs  # segment size in samples

            # Break signal into list of segments in a single-line Python code
            segment1 = audio1[:Div_p]
            segment2 = audio1[Div_p:]
            segments = [segment1, segment2]
            
                    
            #Find out the number of Segments
            #n = len(segments)

            # Process each chunk with your parameters
            # Save each segment in a seperate filename
            for iS, s in enumerate(segments):
                write("audio/data/chunky{0:d}.wav".format(segment_size_t * iS, segment_size_t * (iS + 1)), fs, (s))

            samples = segments[0].get_array_of_samples()
            samples = np.array(samples)
            
            # Calculate Power Spectrum & Cepstrum
            ceps, spec = real_cepstrum(samples, n=None)
            N = samples.shape[0]

            # Power Specturm
            power_spec = np.abs(spec[:N//2])**2

            # Calculate Zero Cross Rate
            frameSize = 256
            overlap = 0
            zcr = ZCR(samples, frameSize, overlap)
            
            index = np.where(power_spec == np.max(power_spec))

            abs_ceps = np.abs(ceps[:N//2])**2
            np.max(abs_ceps)

            #Calculate the spectral Centroid
            
            
            w +=1
            sheet1.write(w, h, int(index[0][0]))
            w +=1
            sheet1.write(w, h, round(np.max(abs_ceps)))
            w +=1 
            sheet1.write(w, h, round(np.average(zcr)))
            
            
            flag = True

wb.save('audio/DataResults.xls')

C:\Users\Dasil\AppData\Local\Temp/ipykernel_20976/565163330.py:56: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, audio1 = read(filename)


TypeError: slice indices must be integers or None or have an __index__ method

In [8]:
split_on_silence?